In [1]:
from dotenv import load_dotenv, find_dotenv

# read local .env file
# find_dotenv 函数通常用于搜索和定位目录树中的 .env 文件，而 load_dotenv 函数用于将 .env 文件中的变量加载到环境中。
_ = load_dotenv(find_dotenv())
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

In [2]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
 

In [3]:
"""
虽然Pydantic / JSON解析器更加强大，但我们最初尝试的数据结构仅具有文本字段。
"""
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
 

In [4]:
format_instructions = output_parser.get_format_instructions()
format_instructions
 

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer to the user\'s question\n\t"source": string  // source used to answer the user\'s question, should be a website.\n}\n```'

In [ ]:
""" output
'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer to the user\'s question\n\t"source": string  // source used to answer the user\'s question, should be a website.\n}\n```'
"""

In [5]:
"""
我们现在得到了一个字符串，其中包含响应应如何格式化的指令，然后我们将其插入到我们的提示中。
"""
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [6]:
_input = prompt.format_prompt(question="what's the capital of france")
output = llm.invoke(_input.to_string())
output

AIMessage(content='```json\n{\n\t"answer": "The capital of France is Paris.",\n\t"source": "https://en.wikipedia.org/wiki/Paris"\n}\n```')

In [ ]:
""" output
AIMessage(content='```json\n{\n\t"answer": "The capital of France is Paris.",\n\t"source": "https://en.wikipedia.org/wiki/Paris"\n}\n```')
"""

In [7]:
output_parser.parse(output.content)
 

{'answer': 'The capital of France is Paris.',
 'source': 'https://en.wikipedia.org/wiki/Paris'}

In [ ]:
""" output
{'answer': 'The capital of France is Paris.',
 'source': 'https://en.wikipedia.org/wiki/Paris'}
"""

In [8]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("answer the users question as best as possible.\n{format_instructions}\n{question}")  
    ],
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)
 

In [9]:
_input = prompt.format_prompt(question="what's the capital of france")
output = llm.invoke(_input.to_messages())
output

AIMessage(content='```json\n{\n\t"answer": "The capital of France is Paris.",\n\t"source": "https://en.wikipedia.org/wiki/Paris"\n}\n```')

In [ ]:
""" output
AIMessage(content='```json\n{\n\t"answer": "The capital of France is Paris.",\n\t"source": "https://en.wikipedia.org/wiki/Paris"\n}\n```')
"""

In [10]:
output_parser.parse(output.content)

{'answer': 'The capital of France is Paris.',
 'source': 'https://en.wikipedia.org/wiki/Paris'}

In [ ]:
""" output
{'answer': 'The capital of France is Paris.',
 'source': 'https://en.wikipedia.org/wiki/Paris'}
"""